In [25]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score
import shutil
from concurrent.futures import ThreadPoolExecutor
import wandb
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy

In [26]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# path = '/content/drive/MyDrive/chexpert_data_v2/train/patient63407'
# print(os.path.exists(path))


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Using device:", device)

 Using device: cuda


In [ ]:
# # ======= LRF =======
# class LRFModel(nn.Module):
#     def __init__(self, backbone_name='vit_base_patch16_224', rank=64, num_classes=5):
#         super().__init__()
#         self.backbone = timm.create_model(backbone_name, pretrained=True)
#         in_features = self.backbone.head.in_features
#         self.backbone.reset_classifier(0)

#         self.low_rank_projection = nn.Sequential(
#             nn.Linear(in_features, rank, bias=False),
#             nn.ReLU()
#         )
#         self.classifier = nn.Linear(rank, num_classes)

#     def forward_features(self, x):
#         return self.backbone.forward_features(x)

#     def forward(self, x):
#         feats = self.forward_features(x)
#         proj = self.low_rank_projection(feats)
#         logits = self.classifier(proj)
#         return logits

#     def get_image_embedding(self, x):
#         feats = self.forward_features(x)
#         proj = self.low_rank_projection(feats)
#         return proj

In [30]:
class BaseModel(nn.Module):
    def __init__(self, backbone_name='vit_base_patch16_224', num_classes=5):
        super().__init__()
        # بارگذاری مدل ViT پایه از timm
        self.backbone = timm.create_model(backbone_name, pretrained=True)

        # حذف لایه classifier اصلی مدل (head)
        in_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)

        # اضافه کردن لایه خطی نهایی با خروجی num_classes
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # استخراج ویژگی‌ها با backbone
        feats = self.backbone.forward_features(x)

        # پیش‌بینی نهایی
        logits = self.classifier(feats)
        return logits

    def get_image_embedding(self, x):
        # فقط استخراج ویژگی‌ها (embedding)
        feats = self.backbone.forward_features(x)
        return feats

In [31]:


class CheXpertDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):

        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_cols = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # استخراج مسیر نسبی تصویر از فایل CSV
        img_rel_path = self.labels_df.iloc[idx]['Path']

        # حذف پیشوند "CheXpert-v1.0-small/" در صورت وجود
        if img_rel_path.startswith("CheXpert-v1.0-small/"):
            img_rel_path = img_rel_path[len("CheXpert-v1.0-small/"):]

        # ساخت مسیر کامل تصویر
        img_path = os.path.join(self.img_dir, img_rel_path)

        # تلاش برای بارگذاری تصویر، و جایگزینی با تصویر سیاه در صورت خطا
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"[WARNING] Could not load image: {img_path} -- {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))  # تصویر جایگزین سیاه

        # اعمال ترنسفورم در صورت وجود
        if self.transform:
            image = self.transform(image)

        # استخراج لیبل‌ها و تبدیل امن به float32 بدون هشدار
        labels = self.labels_df.iloc[idx][self.label_cols]
        labels = labels.infer_objects(copy=False).fillna(0).values.astype('float32')

        return image, labels, img_rel_path


In [32]:
# ====== Transforms ======
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [33]:


# Load CSVs
full_train_df = pd.read_csv('/content/drive/MyDrive/chexpert_data_v2/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/chexpert_data_v2/valid.csv')

# Sample 90% of data
subset_df = full_train_df.sample(frac=0.7, random_state=42).reset_index(drop=True)
subset_df.to_csv("chexpert_30percent.csv", index=False)

# Setup paths
source_root = '/content/drive/MyDrive/chexpert_data_v2'
target_root = '/content/chexpert_data_v2_selected'
image_paths = subset_df['Path'].str.replace('CheXpert-v1.0-small/', '', regex=False).tolist()

# Create target folder
os.makedirs(target_root, exist_ok=True)

# Track missing files
missing_files = []

# Define safe and fast copier
def copy_file_fast(rel_path):
    src = os.path.join(source_root, rel_path)
    dst = os.path.join(target_root, rel_path)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    try:
        if not os.path.exists(dst):  # Skip if already copied
            shutil.copyfile(src, dst)
    except Exception as e:
        missing_files.append(rel_path)

# Run multi-threaded copying
with ThreadPoolExecutor(max_workers=32) as executor:
    list(tqdm(executor.map(copy_file_fast, image_paths), total=len(image_paths)))

# Save missing files log
if missing_files:
    pd.DataFrame({'missing': missing_files}).to_csv("missing_files.csv", index=False)
    print(f"❌ {len(missing_files)} files failed to copy. Saved in missing_files.csv")
else:
    print("✅ All files copied successfully.")


  4%|▍         | 6308/156390 [02:55<1:09:27, 36.01it/s]


KeyboardInterrupt: 

In [10]:
!ls /content/drive/MyDrive/chexpert_data_v2/


train  train.csv  valid  valid.csv


In [11]:
train_dataset = CheXpertDataset(
    "chexpert_30percent.csv",
    "/content/chexpert_data_v2_selected",
    transform
)

val_dataset = CheXpertDataset(
    "/content/drive/MyDrive/chexpert_data_v2/valid.csv",
    "/content/drive/MyDrive/chexpert_data_v2",
    transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [13]:

model = BaseModel().to(device)
criterion = nn.BCEWithLogitsLoss(reduction='none')
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
scaler = GradScaler()

best_auc = 0
early_stop_counter = 0
early_stop_patience = 5
best_model_state = None


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/tmp/ipython-input-13-582192473.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [14]:
wandb.init(
    project="chexpert-lrf-vit",
    name="run-vit-lrf-v1",
    config={
        "lr": 1e-4,
        "batch_size": 128,
        "epochs": 10,
        "model": "ViT + LRF",
        "rank": 64
    }
)

In [15]:
from torch.amp import autocast, GradScaler

scaler = GradScaler(device="cuda")

def train_one_epoch(model, dataloader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    all_labels = []
    all_outputs = []

    for images, labels, _ in tqdm(dataloader, desc="🧠 Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast(device_type='cuda'):
            outputs = model(images)
            outputs = outputs.mean(dim=1)
            probs = torch.sigmoid(outputs)

            mask = (labels != -1).float()
            loss_raw = criterion(outputs, labels)
            loss = (loss_raw * mask).sum() / mask.sum().clamp(min=1.0)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * images.size(0)
        all_labels.append(labels.detach().cpu())
        all_outputs.append(probs.detach().cpu())

    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_outputs).numpy()
    try:
        auc = roc_auc_score(y_true, y_pred, average='macro')
    except:
        auc = None

    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss, auc


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels, _ in tqdm(dataloader, desc="🔍 Validation"):
            images, labels = images.to(device), labels.to(device)

            with autocast(device_type='cuda'):
                outputs = model(images)
                outputs = outputs.mean(dim=1)
                probs = torch.sigmoid(outputs)

                mask = (labels != -1).float()
                loss_raw = criterion(outputs, labels)
                loss = (loss_raw * mask).sum() / mask.sum().clamp(min=1.0)

            total_loss += loss.item() * images.size(0)
            all_labels.append(labels.detach().cpu())
            all_outputs.append(probs.detach().cpu())

    avg_loss = total_loss / len(dataloader.dataset)

    if all_labels and all_outputs:
        try:
            y_true = torch.cat(all_labels).numpy()
            y_pred = torch.cat(all_outputs).numpy()
            auc = roc_auc_score(y_true, y_pred, average='macro')
        except:
            auc = None
    else:
        auc = None

    return avg_loss, auc


In [ ]:
# # نمایش 5 نمونه‌ی تصادفی از train_dataset
# import matplotlib.pyplot as plt

# for i in range(5):
#     img, label, path = train_dataset[i]
#     plt.imshow(img.permute(1, 2, 0))  # برای نمایش [C,H,W] → [H,W,C]
#     plt.title(f"Path: {path}\nLabels: {label}")
#     plt.axis('off')
#     plt.show()


In [16]:
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=20, patience=3, save_path="best_model.pth"):
    scaler = GradScaler()
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(1, epochs + 1):
        print(f"\n📚 Epoch {epoch}/{epochs}")

        train_loss, train_auc = train_one_epoch(model, train_loader, optimizer, criterion, device, scaler)
        val_loss, val_auc = validate(model, val_loader, criterion, device)

        train_auc_str = f" | 📊 Train AUC: {train_auc:.4f}" if train_auc is not None else ""
        val_auc_str = f" | 📈 Val AUC: {val_auc:.4f}" if val_auc is not None else ""

        print(f"✅ Train Loss: {train_loss:.4f}{train_auc_str} | 🧪 Val Loss: {val_loss:.4f}{val_auc_str}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print("💾 Model saved!")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"⚠️ No improvement for {epochs_no_improve} epoch(s).")

            if epochs_no_improve >= patience:
                print("⏹️ Early stopping triggered!")
                break

    print(f"🎯 Training finished. Best Val Loss: {best_val_loss:.4f}")

In [23]:
import os
from PIL import Image

sample_path = 'train/patient44846/study1/view1_frontal.jpg'  # یکی از مسیرهای اروری
img_path = os.path.join('/content/chexpert_data_v2_selected', sample_path)

print("✅ Exists:", os.path.exists(img_path))
if os.path.exists(img_path):
    Image.open(img_path).show()
else:
    print("🚫 مسیر تصویر موجود نیست.")



✅ Exists: False
🚫 مسیر تصویر موجود نیست.


In [24]:
os.path.exists('/content/drive/MyDrive/chexpert_data_v2/train/patient44846/study1/view1_frontal.jpg')


True

In [22]:
# train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=20, patience=3, save_path="/content/best_model.pth")


In [ ]:
# ====== Extract image embeddings and save ======
model.load_state_dict(torch.load("best_model_baseline2.pth"))
model.eval()

image_embeddings = {}
with torch.no_grad():
    for images, _, paths in tqdm(val_loader):
        images = images.to(device)
        embs = model.get_image_embedding(images)  # (B, R)
        for path, emb in zip(paths, embs):
            image_embeddings[path] = emb.cpu()

torch.save(image_embeddings, "/content/drive/MyDrive/image_embeddings.pt")
print("✅ Image embeddings saved!")


100%|██████████| 4/4 [00:04<00:00,  1.01s/it]

✅ Image embeddings saved!


In [ ]:
# import torch
# import gc

# gc.collect()          # جمع‌آوری حافظه آزاد شده در پایتون
# torch.cuda.empty_cache()  # آزادسازی کش حافظه GPU
